### Нарушение согласования

Предыдущая попытка в задаче согласования рода - предсказывать род и сравнивать с ответом pymorphy

Недостаток - большое количество ошибочных предсказаний в словах, таких как "морской". 
Морской порт и цвет морской волны. Род различается, написание - нет.

Попробуем другой подход - предсказание окончаний.

Начнем с поиска ошибки согласования прилагательных

3 типа:
* новЫЙ
* хорошИЙ
* большОЙ

### 1. Построим бандл, сбалансированный по длине предложения и по корпусу.

In [1]:
from tg.grammar_ru.common import Loc
from tg.grammar_ru.corpus import CorpusReader, CorpusBuilder, BucketCorpusBalancer
from tg.grammar_ru.corpus.corpus_reader import read_data

from yo_fluq_ds import Queryable, Query, fluq

from typing import List, Union
from pathlib import Path

import math
import pandas as pd
pd.set_option('display.max_rows', 500)


In [2]:
CORPUS_NAMES = [
    "books.base.zip", 
    # "pub.base.zip"
]
#TODO: add smth else?

CORPUS_LIST = [Loc.corpus_path/corpus_name for corpus_name in CORPUS_NAMES]

In [3]:
CORPUS_LIST

[PosixPath('/home/serhio/Data/1Education/grammar-spring/s2/grammar_ru/data-cache/corpus/books.base.zip')]

In [4]:
def get_sentences_count(corpus_path):
    return sum([
        len(frame.groupby("sentence_id")) 
        for frame in read_data(corpus_path)
    ])

In [29]:
for i, corpus_name in enumerate(CORPUS_NAMES):
    print(f"{corpus_name} = {get_sentences_count(CORPUS_LIST[i])} sentences")

/tmp/ipykernel_10198/3628676302.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) read_data. (Use CorpusReader.read_frames_from_several_corpora)
  for frame in read_data(corpus_path)
100%|██████████| 2533/2533 [00:40<00:00, 61.97it/s] 

books.base.zip = 903668 sentences


In [30]:
LOG_BASE = math.e
BUCKET_PATH = Loc.corpus_path/"prepare/buckets/buckets.parquet"

In [31]:
BucketCorpusBalancer.build_buckets_frame(CORPUS_LIST, BUCKET_PATH, LOG_BASE)

100%|██████████| 2533/2533 [03:12<00:00, 13.16it/s]


In [32]:
pd.read_parquet(BUCKET_PATH)

,sentences,bucket_size
bucket,,
books.base.zip/0,"[2105037, 2635119, 2635122, 2930082, 2930089, ...",129
books.base.zip/1,"[7, 8, 11, 15, 10711, 10730, 10731, 10745, 107...",90591
books.base.zip/2,"[1, 2, 6, 9, 12, 14, 16, 17, 18, 19, 20, 21, 2...",434207
books.base.zip/3,"[0, 3, 4, 5, 10, 13, 10712, 10716, 10718, 1072...",355499
books.base.zip/4,"[10715, 10717, 10724, 10744, 10777, 10793, 107...",23177
books.base.zip/5,"[189579, 784022, 2391783, 5783820, 5998276, 67...",65


In [33]:
BUCKET_NUMBERS = [1, 2, 3, 4]
BUCKET_NUMBERS = [2]
BUCKET_LIMIT = 60_000
BUCKET_LIMIT = 10_000
 #Note: забили на то что в books.base.zip/4 всего 23К предложений.
# некторый дисбаланс все-же остался

In [34]:
BucketCorpusBalancer.filter_buckets_by_bucket_numbers(BUCKET_PATH, BUCKET_NUMBERS)

In [35]:
pd.read_parquet(BUCKET_PATH)

,sentences,bucket_size
books.base.zip/2,"[1, 2, 6, 9, 12, 14, 16, 17, 18, 19, 20, 21, 2...",434207


In [36]:
BALANCED_PATH = Loc.corpus_path/"prepare/balanced/books&pub_60K_balanced.zip"
BALANCED_PATH = Loc.corpus_path/"prepare/balanced/books&pub_tiny_balanced.zip"

In [37]:
from tg.common import Logger
def balancing() -> None:
    balancer = BucketCorpusBalancer(
        buckets = pd.read_parquet(BUCKET_PATH), 
        log_base = LOG_BASE,
        bucket_limit = BUCKET_LIMIT,
    )

    CorpusBuilder.transfuse_corpus(
        sources = CORPUS_LIST,
        destination = BALANCED_PATH,
        selector = balancer
    )

In [38]:
balancing()

/tmp/ipykernel_10198/3137559665.py:3: DeprecationWarning: Call to deprecated class BucketCorpusBalancer. (Use BucketBalances instead)
  balancer = BucketCorpusBalancer(


2023-03-16 23:54:38.308305 INFO: Processed 14 words. 1/2533
2023-03-16 23:54:38.424797 INFO: Processed 52 words. 2/2533
2023-03-16 23:54:38.552183 INFO: Processed 105 words. 3/2533
2023-03-16 23:54:38.683107 INFO: Processed 119 words. 4/2533
2023-03-16 23:54:38.841772 INFO: Processed 161 words. 5/2533
2023-03-16 23:54:38.969134 INFO: Processed 247 words. 6/2533
2023-03-16 23:54:39.096424 INFO: Processed 265 words. 7/2533
2023-03-16 23:54:39.231319 INFO: Processed 336 words. 8/2533
2023-03-16 23:54:39.322947 INFO: Processed 348 words. 9/2533
2023-03-16 23:54:39.434657 INFO: Processed 398 words. 10/2533
2023-03-16 23:54:39.560003 INFO: Processed 426 words. 11/2533
2023-03-16 23:54:39.666774 INFO: Processed 465 words. 12/2533
2023-03-16 23:54:39.780745 INFO: Processed 555 words. 13/2533
2023-03-16 23:54:39.894040 INFO: Processed 592 words. 14/2533
2023-03-16 23:54:40.012576 INFO: Processed 636 words. 15/2533
2023-03-16 23:54:40.141192 INFO: Processed 656 words. 16/2533
2023-03-16 23:54:40

In [19]:
from collections import defaultdict
lengths = defaultdict(int)
for frame in read_data(BALANCED_PATH):
    for corpus_name in CORPUS_NAMES:
        lengths[corpus_name] += len(frame[frame.original_corpus_id == corpus_name].groupby("sentence_id"))

lengths

/tmp/ipykernel_12481/1361944034.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) read_data. (Use CorpusReader.read_frames_from_several_corpora)
  for frame in read_data(BALANCED_PATH):
100%|██████████| 147/147 [00:11<00:00, 12.57it/s]


defaultdict(int, {'books.base.zip': 203177, 'pub.base.zip': 240000})

In [6]:
from tg.grammar_ru import CorpusReader
from tg.grammar_ru.corpus import CorpusBuilder
from pathlib import Path

# corpus = Path('files/corpus.zip')
reader = CorpusReader(BALANCED_PATH)
# reader.get_toc()

In [7]:
# dfs = reader.read_frames().take(10).to_list()

In [34]:
# set('ий его ему им ем ее его ему ее им ем ая ей ую ей ую ие их им ими'.split())
# set('ый ого ому ым ом ое ая ой ую ою ые ых ыми'.split())
# set('ой ого ому им ом ое ого ому ое им ом ая ой ую ой ою ие их им ими'.split())

In [115]:
# for norm_end, group in adjs.groupby('norm_ending'):
#     print(group[['word', 'ending', 'true_ending', 'norm_form', 'norm_ending', 'true_norm_ending']])
#     print('####')

### 2. Построили features для сбалансированного корпуса

In [8]:
from tg.grammar_ru.corpus import CorpusBuilder
from tg.grammar_ru.features import PyMorphyFeaturizer, SlovnetFeaturizer, SyntaxStatsFeaturizer, SyntaxTreeFeaturizer
from pathlib import Path

featurizers = [
    PyMorphyFeaturizer(),
    SlovnetFeaturizer(),
    SyntaxTreeFeaturizer(),
    SyntaxStatsFeaturizer()
]

CorpusBuilder.featurize_corpus(
    Loc.data_cache_path/"corpus/prepare/balanced"/'books&pub_60K_balanced.zip',
    Loc.data_cache_path/"corpus/prepare/balanced"/'books&pub_60K_balanced_feat.zip',
    featurizers
)


### 3. О том, как строим бандл


Существует 3 типа склонения:
 * новЫЙ
 * хорошИЙ
 * большОЙ


Начнем с того, что будем предсказывать окончания только для одного типа склонения.
При этом хотим построить бандл, универсальный для задачи согласования прилагательных.

Отфильтровываем только прилагательные.

pymorphy считает прилагательными такие слова как `один, тот, этот, сей, ваш`

Поэтому отфильтровали еще и mystem'ом. Он различает числительное-прилагательное, местоимение-прилагательное.

Определим тип склонения по окончанию нормальной формы, взятой из pymorphy.

label'ом будет окончание прилагательного.

Составили список возможных окончаний. Берем прилагательное, проверяем на какое из этих окончаний оно заканчивается.

Если ни на какое - ставим Nan, выкидываем из индекса.

Список прилагательных с неопределенными нами окончаниями записали в `temp/undefined_ending.txt`

Кажется, туда попали только
* притяжательные прилагательные. "птичьи", "Надин"
* возвратные причастия. "светящийся"


In [5]:
new = {'ая', 'ого', 'ое', 'ой', 'ом', 'ому',
       'ою', 'ую', 'ые', 'ый', 'ым', 'ыми', 'ых'}

good = {'ая', 'его', 'ее', 'ей', 'ем', 'ему',
        'ие', 'ий', 'им', 'ими', 'их', 'ую', 'яя', 'юю'}

big = {'ая', 'ие', 'им', 'ими', 'их', 'ого',
       'ое', 'ой', 'ом', 'ому', 'ою', 'ую'}

POSSIBLE_ENDINGS = set().union(new, good, big)

In [6]:
len(new)

13

##### Номера окончаний

In [7]:
endings_nums = {e: i for i, e in enumerate(
    sorted(list(POSSIBLE_ENDINGS)))}

endings_nums


{'ая': 0,
 'его': 1,
 'ее': 2,
 'ей': 3,
 'ем': 4,
 'ему': 5,
 'ие': 6,
 'ий': 7,
 'им': 8,
 'ими': 9,
 'их': 10,
 'ого': 11,
 'ое': 12,
 'ой': 13,
 'ом': 14,
 'ому': 15,
 'ою': 16,
 'ую': 17,
 'ые': 18,
 'ый': 19,
 'ым': 20,
 'ыми': 21,
 'ых': 22,
 'юю': 23,
 'яя': 24}

In [8]:
new_declination_labels = {num for e, num in endings_nums.items() if e in new}
new_declination_labels

{0, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22}

##### Загрузка бандла

In [9]:
from tg.common import DataBundle
from tg.common.ml.batched_training import IndexedDataBundle
from tg.grammar_ru.components.plain_context_builder import PlainContextBuilder

# db = DataBundle.load(Loc.data_cache_path/'bundles/agreement/toy')
db = DataBundle.load(Loc.data_cache_path/'bundles/agreement/tiny_full')
db['index'] = db.index[db.index.label.isin(new_declination_labels)]
db = db.copy()

idb = IndexedDataBundle(db.index, db)

In [10]:
idb.index_frame.label.nunique()

12

In [11]:
db.index

,word_id,sentence_id,label,split
sample_id,,,,
0,29,3,12,train
1,52,6,19,display
2,74,8,11,train
3,88,10,20,display
4,93,10,21,display
...,...,...,...,...
2594,83516,54067,0,display
2595,83523,54068,0,display
2596,83555,54072,20,train


In [12]:
db.index.shape

(2078, 4)

In [13]:
sorted(db.index.label.unique())

[0, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22]

In [14]:
db

{'index': {'shape': (2078, 4), 'index_name': 'sample_id'}, 'pymorphy': {'shape': (83730, 16), 'index_name': 'word_id'}, 'slovnet': {'shape': (83730, 17), 'index_name': 'word_id'}, 'src': {'shape': (83730, 19), 'index_name': None}, 'syntax_closure': {'shape': (115785, 4), 'index_name': 'entry_id'}, 'syntax_fixes': {'shape': (83730, 4), 'index_name': 'word_id'}, 'syntax_stats': {'shape': (83730, 6), 'index_name': 'word_id'}}

#### Оставили в индексе только прилагательный 1-го типа склонения. "Новый"

In [15]:
# db.index = db.index[db.index.label.isin(new_declination_labels)]
# db.index = db.index[db.index.label==0]

In [16]:
sorted(db.index.label.unique())


[0, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22]

In [17]:
new_declination_labels

{0, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22}

In [18]:
new_declination_labels - set(db.index.label)

{16}

In [19]:
plain_context_builder = PlainContextBuilder(include_zero_offset=False,
                          left_to_right_contexts_proportion=0.5)
contexts = plain_context_builder.build_context(idb, context_size=6).sort_index()

In [20]:
from tg.grammar_ru.components import PlainContextBuilder

context_builder = PlainContextBuilder(
    include_zero_offset=True,
    left_to_right_contexts_proportion=0.5
)

In [21]:
from tg.common.ml.batched_training import context as btc
from tg.grammar_ru.components import CoreExtractor

def create_assembly_point(context_length = 6):
    ap = btc.ContextualAssemblyPoint(
        name = 'features',
        context_builder = context_builder,
        extractor = CoreExtractor(join_column='another_word_id'),
        context_length=context_length
    )
    ap.reduction_type = ap.reduction_type.Dim3Folded
    return ap

In [22]:
ap = create_assembly_point()
ap.hidden_size = 50
ap.dim_3_network_factory.network_type = btc.Dim3NetworkType.LSTM
head_factory = ap.create_network_factory()
# head = head_factory(batch)

In [23]:
import torch


def _update_sizes_with_argument(argument_name, argument, sizes, modificator):
    if argument is None:
        return sizes
    elif isinstance(argument, torch.Tensor):
        return modificator(sizes, argument.shape[1])
    elif isinstance(argument, pd.DataFrame):
        return modificator(sizes, argument.shape[1])
    elif isinstance(argument, int):
        return modificator(sizes, argument)
    else:
        raise ValueError(
            f"Argument {argument_name} is supposed to be int, Tensor or none, but was `{argument}`")


class FullyConnectedNetwork(torch.nn.Module):
    def __init__(self,
                 sizes: List[int],
                 input: Union[None, torch.Tensor, int] = None,
                 output: Union[None, torch.Tensor, int] = None):
        super(FullyConnectedNetwork, self).__init__()
        sizes = _update_sizes_with_argument(
            'input', input, sizes, lambda s, v: [v] + s)
        sizes = _update_sizes_with_argument(
            'output', output, sizes, lambda s, v: s + [v])
        self.layers = torch.nn.ModuleList()
        for i in range(len(sizes) - 1):
            self.layers.append(torch.nn.Linear(sizes[i], sizes[i + 1]))

    def forward(self, input):
        X = input
        for layer in self.layers:
            X = layer(X)
            X = torch.sigmoid(X)
        return X

In [24]:
import torch
from tg.common.ml.batched_training import factories as btf


class Network(torch.nn.Module):
    def __init__(self, head, hidden_size, batch):
        super(Network, self).__init__()
        self.head = head
        self.tail = FullyConnectedNetwork(
            sizes=[], input=hidden_size, output=batch.index_frame.label.nunique())
        # TODO don't hard code output

    def forward(self, batch):
        return self.tail(self.head(batch))


class NetworkFactory:
    def __init__(self, assembly_point):
        self.assembly_point = assembly_point

    def __call__(self, batch):
        head_factory = self.assembly_point.create_network_factory()
        head = head_factory(batch)
        return Network(head, self.assembly_point.hidden_size,  batch)


network_factory = NetworkFactory(ap)


In [25]:
from tg.common.ml import batched_training as bt

class MulticlassMetrics(bt.Metric):
    def __init__(self, add_accuracy=True, add_rating=False):
        self.add_accuracy = add_accuracy
        self.add_rating = add_rating

    def get_names(self):
        result = []
        if self.add_accuracy:
            result.append('accuracy')
        if self.add_rating:
            result.append('rating')
        return result

    def measure(self, df, _):
        prefix = 'true_label_'
        labels = []
        for c in df.columns:
            if c.startswith(prefix):
                labels.append(c.replace(prefix, ''))

        def ustack(df, prefix, cols, name):
            df = df[[prefix+c for c in cols]]
            df.columns = [c for c in cols]
            df = df.unstack().to_frame(name)
            return df

        predicted = ustack(df, 'predicted_label_', labels, 'predicted')
        true = ustack(df, 'true_label_', labels, 'true')
        df = predicted.merge(true, left_index=True,
                             right_index=True).reset_index()
        df.columns = ['label', 'sample', 'predicted', 'true']
        df = df.feed(fluq.add_ordering_column(
            'sample', ('predicted', False), 'predicted_rating'))

        match = (df.loc[df.predicted_rating ==
                 0].set_index('sample').true > 0.5)
        rating = df.loc[df.true > 0.5].set_index('sample').predicted_rating
        result = []
        if self.add_accuracy:
            result.append(match.mean())
        if self.add_rating:
            result.append(rating.mean())
        return result

In [26]:
from sklearn.metrics import roc_auc_score
import pandas as pd
from tg.common.ml import batched_training as bt
from tg.common.ml import dft


def get_multilabel_extractor():
    label_extractor = (bt.PlainExtractor
                       .build(btf.Conventions.LabelFrame)
                       .index()
                       .apply(take_columns=['label'],
                              transformer=dft.DataFrameTransformerFactory.default_factory())
                       )
    return label_extractor


class TrainingTask(btf.TorchTrainingTask):
    def __init__(self):
        super(TrainingTask, self).__init__()
        self.metric_pool = bt.MetricPool().add(MulticlassMetrics())
        self.features_ap = create_assembly_point()

    def initialize_task(self, idb):
        self.setup_batcher(
            idb, [ap.create_extractor(), get_multilabel_extractor()])
        self.setup_model(network_factory, ignore_consistancy_check=True)


task = TrainingTask()
task.settings.epoch_count = 2
result = task.run(db)


2023-03-17 01:02:44.142475 INFO: Training starts. Info: {}
2023-03-17 01:02:44.145514 INFO: Ensuring/loading bundle. Bundle before:
{'index': {'shape': (2078, 4), 'index_name': 'sample_id'}, 'pymorphy': {'shape': (83730, 16), 'index_name': 'word_id'}, 'slovnet': {'shape': (83730, 17), 'index_name': 'word_id'}, 'src': {'shape': (83730, 19), 'index_name': None}, 'syntax_closure': {'shape': (115785, 4), 'index_name': 'entry_id'}, 'syntax_fixes': {'shape': (83730, 4), 'index_name': 'word_id'}, 'syntax_stats': {'shape': (83730, 6), 'index_name': 'word_id'}}
2023-03-17 01:02:44.148147 INFO: Bundle loaded
{'index': {'shape': (2078, 4), 'index_name': 'sample_id', 'columns': ['word_id', 'sentence_id', 'label', 'split'], 'index': [0, 1, 2, 3, 4, '...']}, 'pymorphy': {'shape': (83730, 16), 'index_name': 'word_id', 'columns': ['normal_form', 'alternatives', 'score', 'delta_score', 'POS', '...'], 'index': [0, 1, 2, 3, 4, '...']}, 'slovnet': {'shape': (83730, 17), 'index_name': 'word_id', 'columns':